# Olist Product Recommendation System
### Part 1 Data collection and Data Cleaning 
#### Author: Olabisi Sunmon | 10th April 2023

### Problem Statement

How can we create a customised product recommendation system using data analysis and machine learning techniques to help Olist customers discover new products and find relevant items for purchase, to boost revenue and customer purchase rates.

-------
In the notebook I will be importing the raw database from Kaggle and conducting cleaning.

----------
### Data Source 

The Olist Store provides a dataset of Brazilian ecommerce transactions, consisting of 100,000 orders from 2016 to 2018 made on various marketplaces in the country. This data consists of 8 tables with an extra dataset containing English translations.

The Data can be sourced from;https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce




In [1]:
#Import packages 
import numpy as np
import pandas as pd
import joblib 

#### Import Files that have been downloaded from Kaggle.

In [2]:
# Load the Data
df_main = pd.read_csv('data/raw/olist_orders_dataset.csv')
df_customer = pd.read_csv('data/raw/olist_customers_dataset.csv')
df_geo = pd.read_csv('data/raw/olist_geolocation_dataset.csv')
df_items = pd.read_csv('data/raw/olist_order_items_dataset.csv')
df_payments = pd.read_csv('data/raw/olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('data/raw/olist_order_reviews_dataset.csv')
df_products = pd.read_csv('data/raw/olist_products_dataset.csv')
df_sellers = pd.read_csv('data/raw/olist_sellers_dataset.csv')
df_product_English = pd.read_csv('data/raw/product_category_name_translation.csv')

### Data Dictionary

---

**Customer Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`customer_id` | key to the orders dataset. Each order has a unique customer_id      | Categorical     
`customer_unique_id`    | unique identifier of a customer.      | Categorical  
`customer_zip_code` | first five digits of customer zip code | Numeric
`customer_city` | customer city name | Categorical 
`customer_state` | customer state | Categorical 


**Geo Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`geolocation_zip_code_prefix` | latitude    | Numeric     
`geolocation_lat`    | longitude     | Numeric 
`geolocation_lng` | first five digits of customer zip code | Numeric
`geolocation_city` | city name | Categorical 
`geolocation_state` | state| Categorical 


**Items Data:**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`order_id` | order unique identifier   | Categorical     
`order_item_id`    | sequential number identifying number of items included in the same order.    | Numeric 
`product_id` | product unique identifier | Categorical  
`seller_id` | seller unique identifier | Categorical 
`shipping_limit_date` | Shows the seller shipping limit date for handling the order over to the logistic partner.| Categorical 
`price` | item price | Numeric 
`freight_value` |item freight value item (if an order has more than one item the freight value is splitted between items).|  Numeric 


**Payment Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`order_id` | unique identifier of an order.    | Categorical    
`payment_sequential`    | a customer may pay an order with more than one payment method. If he does so, a sequence will be created to accommodate all     | Numeric 
`payment_type` | first five digits of customer zip code | Numeric
`payment_installments` | number of installments chosen by the customer. | Numeric
`payment_value` | transaction value.| Numeric


**Reviews Data:**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`review_id` | unique review identifier.   | Categorical     
`order_id`    | unique order identifier.    | Categorical 
`review_score`    | Note ranging from 1 to 5 given by the customer on a satisfaction survey.   | Numeric 
`review_comment_title` | Comment title from the review left by the customer, in Portuguese. | Categorical
`review_comment_message` | Comment message from the review left by the customer, in Portuguese. | Categorical 
`review_creation_date` | Shows the date in which the satisfaction survey was sent to the customer.| Numeric 
`review_answer_timestamp` | Shows satisfaction survey answer timestamp. | Numeric 


**Main Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`order_id` | unique identifier of an order.    | Categorical    
`customer_id`    | key to the customer dataset. Each order has a unique customer_id.  | Categorical  
`order_status` | Reference to the order status (delivered, shipped, etc). | Categorical  
`order_purchase_timestamp` | Shows the purchase timestamp. | Numeric
`order_approved_at` | Shows the payment approval timestamp.| Numeric
`order_delivered_carrier_date` | Shows the order posting timestamp. When it was handled to the logistic partner. | Numeric
`order_delivered_customer_date` | Shows the actual order delivery date to the customer.| Numeric
`order_estimated_delivery_date` |Shows the estimated delivery date that was informed to customer at the purchase moment.| Numeric

**Product Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`product_id` | unique product identifier.   | Categorical    
`product_category_name`    | root category of product, in Portuguese.    | Categorical   
`Product_name_lenght` | number of characters extracted from the product name. | Numeric
`product_description_lenght` | number of characters extracted from the product description. | Numeric
`product_photos_qty` | number of product published photos| Numeric
`product_weight_g` | product weight measured in grams. | Numeric
`product_length_cm` | `product_length_cm` | number of characters extracted from the product description. | Numeric
`product_height_cm` | product height measured in centimeters.| Numeric
`product_width_cm` | product width measured in centimeters.| Numeric

**Sellers Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`seller_id` | seller unique identifier   | Categorical    
`seller_zip_code_prefix`    |first 5 digits of seller zip code    | Numeric 
`seller_city` | seller city name| Categorical
`seller_state` | seller state | Categorical

**Product English Data :**

Column Name | Data Description | Data Type |
---------------- |-----------       |-------    
`product_category_name` | category name in Portuguese   | Categorical    
`product_category_name_english`    |category name in English   | Categorical    




### Inspecting and cleaning Data 

In [3]:
#Function to review dataframes
def data_overview(df):

    display(df.head())
    print(df.info())
    print('')
    print("NA's present in the DF per column:")
    print('')
    print(df.isna().sum())
    display(f'There are {df.duplicated().sum()} duplicated rows')
    display(f'The shape of the DF is {df.shape}')
   

In [4]:
data_overview(df_main)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

NA's present in the DF per column:

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date  

'There are 0 duplicated rows'

'The shape of the DF is (99441, 8)'

#### Convert to date time

In [5]:
df_main['order_purchase_timestamp'] = pd.to_datetime(df_main['order_purchase_timestamp'])
df_main['order_approved_at'] = pd.to_datetime(df_main['order_approved_at'])
df_main['order_delivered_carrier_date'] = pd.to_datetime(df_main['order_delivered_carrier_date'])
df_main['order_delivered_customer_date'] = pd.to_datetime(df_main['order_delivered_customer_date'])
df_main['order_estimated_delivery_date'] = pd.to_datetime(df_main['order_estimated_delivery_date'])



In [6]:
#Addressing Na's 
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


df_main have a significant number of missing values, as I do not need these columns for analysis or modeling I will drop them

In [7]:
df_main = df_main.drop(['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'], axis=1)

In [8]:
data_overview(df_customer)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None

NA's present in the DF per column:

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


'There are 0 duplicated rows'

'The shape of the DF is (99441, 5)'

No cleaning necessary for df_customer.

In [9]:
data_overview(df_geo)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB
None

NA's present in the DF per column:

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64


'There are 261831 duplicated rows'

'The shape of the DF is (1000163, 5)'

I will Delete all 261831 duplications as no new infomation is gained by keeping the duplicates.

In [10]:
df_geo = df_geo.drop_duplicates()

In [11]:
data_overview(df_items)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None

NA's present in the DF per column:

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


'There are 0 duplicated rows'

'The shape of the DF is (112650, 7)'

No cleaning necessary for df_items.

In [12]:
df_items['shipping_limit_date'] = pd.to_datetime(df_items['shipping_limit_date'])

In [13]:
data_overview(df_payments)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
None

NA's present in the DF per column:

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64


'There are 0 duplicated rows'

'The shape of the DF is (103886, 5)'

In [14]:
No cleaning necessary for df_payments.

SyntaxError: invalid syntax (3789416559.py, line 1)

In [ ]:
data_overview(df_product_English)

In [ ]:
No cleaning necessary for df_product_English.

In [ ]:
data_overview(df_products)


In [ ]:
df_products.isna().sum().value_counts 

There is a significant number of missing values in df_products, I will not any of these columns besides `product_category_name` for my analysis and modeling so I will drop the others.

In [ ]:
#Dropping columns
df_products = df_products.drop(['product_name_lenght', 'product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm'], axis=1)

In [ ]:
na = df_products[df_products['product_category_name'].isna()] # Create a DataFrame with Na values 

In [ ]:
na.groupby('product_id')[['product_category_name']].sum() #Display

In [ ]:
df_products['product_category_name'].nunique

All the 610 na values in the `product_category_name` column are unique products, as all 32951 `product_id`'s are unique. So I will not be able to locate the corresponding `product_category_name` in the df. Therfeore I will not be addressing the Na values in the `product_category_name` as I have no direct mapping. I will not be dropping the column as I will be using it for analysis and modelling.

In [ ]:
data_overview(df_sellers)

In [ ]:
data_overview(df_items)

In [ ]:
No cleaning necessary for df_items.

In [ ]:
data_overview(df_reviews)

df_reviews has a significant number of missing values, I will not need these columns for analysis or modeling so I will drop them.

In [ ]:
df_reviews = df_reviews.drop(['review_comment_title', 'review_comment_message'], axis=1)

#### The dataFrames are now ready for EDA

In [ ]:
#Export Clean Dataset 

joblib.dump(df_customer,'data/Clean/customer_clean.pkl')
joblib.dump(df_geo,'data/Clean/geo_clean.pkl')
joblib.dump(df_items,'data/Clean/items_clean.pkl')
joblib.dump(df_main,'data/Clean/main_clean.pkl')
joblib.dump(df_products,'data/Clean/products_clean.pkl')
joblib.dump(df_payments,'data/Clean/payments_clean.pkl')
joblib.dump(df_sellers,'data/Clean/sellers_clean.pkl')
joblib.dump(df_reviews,'data/Clean/reviews_clean.pkl')
joblib.dump(df_product_English,'data/Clean/product_english_clean.pkl')
